In [1]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import numpy as np
import pandas as pd
# print(pd.__version__)

In [92]:
sQuizz="10"
nQuizz=int(sQuizz)
fCM=u"/Users/gilles/sDrive/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/sDrive/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
fQuizzoodle=u"/Users/gilles/sDrive/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-%s.csv"%sQuizz
# fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/24-Quizz/Quizz-%s.csv"%sQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")
print fCM,fQuizzoodle

/Users/gilles/sDrive/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv /Users/gilles/sDrive/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-10.csv


In [93]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Lecture des données

In [94]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
# alias

In [95]:
len(alias)

189

In [96]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(fQuizzoodle,sep=";",encoding="utf8",header=0,names=colQuizz)
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [97]:
lQuizz, dfCM

(                pseudo  points   temps
 0    Desmouliès Eloane       4  208.29
 1            Bel elena       4  248.96
 2          noemlatorre       4  255.65
 3            Anaé Lamy       3  115.31
 4        Noémie Nelson       3  138.57
 5    Bouffenie marylou       3  171.78
 6     PalomaPigeonnier       3  171.98
 7       Eloise Belvisi       3  191.97
 8        Maelys bertel       3  201.61
 9              Flavroy       3  216.00
 10           Couplaura       3  216.96
 11       Chloé Roubert       3  222.02
 12         Jubertlouna       3  250.00
 13         Elisalatour       3  250.96
 14   degermayquitterie       3  253.30
 15            ayanahla       2   72.23
 16             orlanne       2   77.81
 17  camille etcheverry       2  120.66
 18          Wu Qinglin       2  132.45
 19        Esteves Emma       2  137.46
 20        JustineGanne       2  138.97
 21   roomane.broucaret       2  149.39
 22       Guerbasaldjia       2  151.07
 23       calista veral       2  153.63


In [98]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

Couplaura
Wu Qinglin
roomane.broucaret
gabrelle sartori
Lola Puyo
Lou.hyd


# Listage des alias

In [99]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()
dfCM["P-N"]=dfCM[u"prénom"].str.lower().str.strip()+"-"+dfCM[u"nom"].str.lower().str.strip()
dfCM["N-P"]=dfCM[u"nom"].str.lower().str.strip()+"-"+dfCM[u"prénom"].str.lower().str.strip()
dfCM["P.N"]=dfCM[u"prénom"].str.lower().str.strip()+"."+dfCM[u"nom"].str.lower().str.strip()
dfCM["N.P"]=dfCM[u"nom"].str.lower().str.strip()+"."+dfCM[u"prénom"].str.lower().str.strip()
# dfCM

In [100]:
pn=dfCM["PN"].tolist()
pnT=dfCM["P-N"].tolist()
pnD=dfCM["P.N"].tolist()
np=dfCM["NP"].tolist()
npT=dfCM["N-P"].tolist()
npD=dfCM["N.P"].tolist()
lNomPrenom={"PN":pn,"P-N":pnT,"P.N":pnD,"NP":np,"N-P":npT,"N.P":npD}

In [101]:
for pseudo in pseudos:
    for nomPrenom in lNomPrenom:
        if unidecode.unidecode(pseudo.lower()) in lNomPrenom[nomPrenom]:
            print pseudo,dfCM[dfCM[nomPrenom]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
            alias[pseudo]=[str(dfCM[dfCM[nomPrenom]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]

Wu Qinglin 449075
Lola Puyo 441166


In [102]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

Couplaura
roomane.broucaret
gabrelle sartori
Lou.hyd


In [103]:
for nP,pseudoBrut in enumerate(pseudos):
    pseudo=unidecode.unidecode(pseudoBrut)
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudoBrut]=idPseudo

Couplaura


Nom a chercher:
 laura


               PN      id
22  laura coupama  400648


id :
 400648


roomane.broucaret


...

id :
 449461


Lou.hyd


Nom a chercher:
 lou


                        PN      id
11       marylou bouffenie  446920
24   louise dague--brandao  448118
33        maissa el haloui  446773
52               lou hydie  448277
54            louna jubert  448621
78         louwen moronval  438823
110      lou thibaut--remi  450446


id :
 448277


In [104]:
# pseudo=u"csaintavi001 ebismes001"
# alias[pseudo]=["415962","413429"]

In [105]:
sPseudo="bisme"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [106]:
pseudos

[u'Couplaura', u'roomane.broucaret', u'gabrelle sartori', u'Lou.hyd']

In [107]:
# alias["Sofiane"]

In [108]:
for al in alias:
    if not isinstance(alias[al],list):
        if " " in alias[al]:
            alias[al]=alias[al].split(" ")
        else:
            alias[al]=[alias[al]]

In [109]:
len(alias)

195

In [110]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'A.verdel': ['447703'],
  'ANDHERIN': ['448262'],
  'ANDRIA.HERIN': ['448262'],
  'Alex.verdel': ['447703'],
  'Alice Alonso': ['450977'],
  'Alix Lonlas': ['450067'],
  u'Am\xe9lie Lauilh\xe9': ['448793'],
  'Ana PIOTON': ['451262'],
  'Ana Pioton': ['451262'],
  u'Ana\xe9': ['438682'],
  u'Ana\xe9 Lamy': ['438682'],
  'Andria.Heriniaina': ['448262'],
  'Apay merv': ['450285'],
  'Apaydin Mervan': ['450285'],
  'Bel Elena-Rose': ['416094'],
  'Bel elena': ['416094'],
  'Belvisi Eloise': ['450742'],
  'Blanche Navarro': ['451482'],
  'Bouffenie marylou': ['446920'],
  u'CL\xc9MENT.augez': ['445328'],
  'Camilla Sulejmani': ['402804'],
  'Camille Etcheverry': ['446690'],
  'Charlotte Durocher': ['444843'],
  'Cheick GUIRA': ['451579'],
  'Chlo Roub': ['446964'],
  u'Chlo\xe9 Roubert': ['446964'],
  u'Chlo\xe9 Roubert Dieuaide': ['446964'],
  'Choquet Clara': ['448189'],
  'Choquet clara': ['448189'],
  'Cla cho': ['448189'],
  'Clara Choquet': ['448189'],
  'Clara choquet

In [111]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Calcul des notes
Attention au nombre de questions

In [112]:
nbQuestions=[7,7,7,7,7,7,7,6,6,6,6]
nbQuestions=[7,7,7,7,7,7,6,6,6,6,6]
nbQ=float(nbQuestions[nQuizz-1])
print nbQ

6.0


In [113]:
def aliasQuizz(pseudo):
    result=None
    if pseudo in alias:
        result=alias[pseudo]
    return result

def noteQuizz(points):   
    result=8
    if points>0:
        result=int(round(10*(1+points/nbQ),0))
    return result
        
        

lQuizz["id"]=lQuizz["pseudo"].apply(aliasQuizz)
lQuizz["note"]=lQuizz["points"].apply(noteQuizz)

In [114]:
def splitPseudo1(id):
    result=None
    if id and len(id)>0:
        result=id[0]
    return result

def splitPseudo2(id):
    result=None
    if id and len(id)>1:
        result=id[1]
    return result

def splitPseudo3(id):
    result=None
    if id and len(id)>2:
        result=id[2]
    return result

lQuizz["id1"]=lQuizz["id"].apply(splitPseudo1)
lQuizz["id2"]=lQuizz["id"].apply(splitPseudo2)
lQuizz["id3"]=lQuizz["id"].apply(splitPseudo3)

lQuizz

,pseudo,points,temps,id,note,id1,id2,id3
0,Desmouliès Eloane,4,208.29,[446480],17,446480,None,None
1,Bel elena,4,248.96,[416094],17,416094,None,None
2,noemlatorre,4,255.65,[447541],17,447541,None,None
3,Anaé Lamy,3,115.31,[438682],15,438682,None,None
4,Noémie Nelson,3,138.57,[446682],15,446682,None,None
5,Bouffenie marylou,3,171.78,[446920],15,446920,None,None
6,PalomaPigeonnier,3,171.98,[450546],15,450546,None,None
7,Eloise Belvisi,3,191.97,[450742],15,450742,None,None
8,Maelys bertel,3,201.61,[450554],15,450554,None,None
9,Flavroy,3,216.00,[451874],15,451874,None,None


In [115]:
df1=lQuizz[["pseudo","note","temps","id1"]].dropna()
df2=lQuizz[["pseudo","note","temps","id2"]].dropna()
df3=lQuizz[["pseudo","note","temps","id3"]].dropna()
df1.columns=["pseudo","note","temps","id"]
df2.columns=["pseudo","note","temps","id"]
df3.columns=["pseudo","note","temps","id"]

In [116]:
df123=pd.concat([df1,df2,df3]).sort_values(["note","temps"],ascending=[False,True]).reset_index(drop=True)
df123

,pseudo,note,temps,id
0,Desmouliès Eloane,17,208.29,446480
1,Bel elena,17,248.96,416094
2,noemlatorre,17,255.65,447541
3,Anaé Lamy,15,115.31,438682
4,Noémie Nelson,15,138.57,446682
5,Bouffenie marylou,15,171.78,446920
6,PalomaPigeonnier,15,171.98,450546
7,Eloise Belvisi,15,191.97,450742
8,Maelys bertel,15,201.61,450554
9,Flavroy,15,216.00,451874


In [117]:
df123.loc[(df123.note==8)&(df123.temps>0),"note"]=10

In [118]:
df123.loc[1,"note"]

17

In [119]:
df123.at[0, 'note'] = 20
if df123.loc[1, "note"]<18:
    df123.at[1, 'note'] = 18
df123

,pseudo,note,temps,id
0,Desmouliès Eloane,20,208.29,446480
1,Bel elena,18,248.96,416094
2,noemlatorre,17,255.65,447541
3,Anaé Lamy,15,115.31,438682
4,Noémie Nelson,15,138.57,446682
5,Bouffenie marylou,15,171.78,446920
6,PalomaPigeonnier,15,171.98,450546
7,Eloise Belvisi,15,191.97,450742
8,Maelys bertel,15,201.61,450554
9,Flavroy,15,216.00,451874


In [120]:
dfNotes=df123.groupby("id").max().sort_values("note",ascending=False)
dfNotes.to_csv(fNotes,sep="\t",encoding="utf8")

In [121]:
dfNotes

,pseudo,note,temps
id,,,
446480,Desmouliès Eloane,20,208.29
416094,Bel elena,18,248.96
447541,noemlatorre,17,255.65
451874,Flavroy,15,216.00
450554,Maelys bertel,15,201.61
448621,Jubertlouna,15,250.00
446964,Chloé Roubert,15,222.02
446920,Bouffenie marylou,15,171.78
450505,degermayquitterie,15,253.30
